In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

prompt = "'<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\nJanet의 오리는 하루에 16개의 알을 낳습니다. 그녀는 매일 아침으로 3개를 먹고, 친구들을 위해 머핀을 구울 때 4개를 사용합니다. 남은 계란은 매일 농산물 시장에서 신선한 오리 알 하나당 2달러에 판매합니다. 그녀는 매일 농산물 시장에서 얼마를 버나요?<|im_end|>\n<|im_start|>assistant\n"

# Qwen2.5 모델과 토크나이저 로드
model_name = "./final_model"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map='cuda:0', torch_dtype=torch.bfloat16)

initial_memory = torch.cuda.memory_allocated()
print(f"초기 GPU 메모리 사용량: {initial_memory / 1024**2:.2f} MB")

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


초기 GPU 메모리 사용량: 2952.71 MB


In [2]:
# 입력 텍스트 토큰화
device = 'cuda:0'
inputs = tokenizer(prompt, return_tensors="pt")
inputs = {k: v.to(device) for k, v in inputs.items()}
max_new_tokens = 256

# 텍스트 생성
with torch.no_grad():
    outputs = model.generate(
        inputs["input_ids"],
        max_new_tokens=max_new_tokens,
        temperature=0.5,
        top_p=0.3,
        do_sample=True
    )

# 생성된 텍스트 디코딩
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
memory = torch.cuda.memory_allocated()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [3]:
print(f"{generated_text}")

'system
You are a helpful assistant.
user
Janet의 오리는 하루에 16개의 알을 낳습니다. 그녀는 매일 아침으로 3개를 먹고, 친구들을 위해 머핀을 구울 때 4개를 사용합니다. 남은 계란은 매일 농산물 시장에서 신선한 오리 알 하나당 2달러에 판매합니다. 그녀는 매일 농산물 시장에서 얼마를 버나요?
assistant
12.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.000.0
